In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy as np
import pickle
from glob import glob
from tqdm.notebook import tqdm, trange

"""Change to the data folder"""
new_path = "./new_train/new_train"

cuda_status = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

### Create a loader to enable batch processing

In [17]:
import math

batch_sz = 64

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [np.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [np.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    car_mask = [scene['car_mask'] for scene in batch]
    
    inp = torch.FloatTensor(inp)
    out = torch.LongTensor(out)
    scene_ids = torch.LongTensor(scene_ids)
    car_mask = torch.LongTensor(car_mask)
    
    num_cars = np.zeros((inp.shape[0]))
    offsets = np.zeros((inp.shape[0], 2))
    
    real_input = np.zeros((inp.shape[0], 7, 19, 4))
    real_output = np.zeros((inp.shape[0], 7, 30, 2))
    
    for i in range(inp.shape[0]):
        
        num_vehicles = 0
        for j in range(60):
            if car_mask[i][j][0] == 1:
                num_vehicles += 1
        num_cars[i] = num_vehicles
        
        agent_id = agent_ids[i]
        vehicle_index = 0
        found = False
        while not found:
            if track_ids[i][vehicle_index][0][0] == agent_id:
                found = True
            else:
                vehicle_index += 1
        start_x = inp[i][vehicle_index][0][0]
        start_y = inp[i][vehicle_index][0][1]
        
        offsets[i][0] = start_x
        offsets[i][1] = start_y
        
        inp[i,0:num_vehicles,:,0] -= start_x
        inp[i,0:num_vehicles,:,1] -= start_y
        
        distances = [0] * num_vehicles
        for j in range(num_vehicles):
            if j == vehicle_index:
                continue
            distances[j] = math.sqrt(((inp[i,j,0,0] - inp[i,vehicle_index,0,0])**2) + ((inp[i,j,0,1] - inp[i,vehicle_index,0,1])**2))
        
        included = []
        for j in range(6):
            min_distance = -1
            min_index = -1
            
            for k in range(len(distances)):
                if k == vehicle_index:
                    continue
                
                if min_index == -1 and k not in included:
                    min_distance = distances[k]
                    min_index = k
                elif k not in included and distances[k] < min_distance:
                    min_distance = distances[k]
                    min_index = k
            if min_index == -1:
                real_input[i][j] = np.zeros((19,4))
            else:
                included.append(min_index)
                real_input[i][j] = inp[i,min_index,:,:]
            real_input[i][j] = inp[i,min_index,:,:]
            real_output[i][j] = out[i,min_index,:,0:2]
        real_input[i][6] = inp[i,vehicle_index,:,:]
        real_output[i] = out[i,vehicle_index,:,0:2]
    
    real_input = torch.LongTensor(real_input)
    real_output = torch.LongTensor(real_output)
    offsets = torch.LongTensor(offsets)
    num_cars = torch.LongTensor(num_cars)
    return [real_input, real_output, scene_ids, offsets]

def test_collate(batch):
    inp = [np.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    car_mask = [scene['car_mask'] for scene in batch]
    
    inp = torch.FloatTensor(inp)
    #out = torch.LongTensor(out)
    scene_ids = torch.LongTensor(scene_ids)
    car_mask = torch.LongTensor(car_mask)
    
    num_cars = np.zeros((inp.shape[0]))
    offsets = np.zeros((inp.shape[0], 2))
    
    real_input = np.zeros((inp.shape[0], 7, 19, 4))
    real_output = np.zeros((inp.shape[0], 7, 30, 2))
    
    for i in range(inp.shape[0]):
        
        num_vehicles = 0
        for j in range(60):
            if car_mask[i][j][0] == 1:
                num_vehicles += 1
        num_cars[i] = num_vehicles
        
        agent_id = agent_ids[i]
        vehicle_index = 0
        found = False
        while not found:
            if track_ids[i][vehicle_index][0][0] == agent_id:
                found = True
            else:
                vehicle_index += 1
        start_x = inp[i][vehicle_index][0][0]
        start_y = inp[i][vehicle_index][0][1]
        
        offsets[i][0] = start_x
        offsets[i][1] = start_y
        
        inp[i,0:num_vehicles,:,0] -= start_x
        inp[i,0:num_vehicles,:,1] -= start_y
        
        distances = [0] * num_vehicles
        for j in range(num_vehicles):
            if j == vehicle_index:
                continue
            distances[j] = math.sqrt(((inp[i,j,0,0] - inp[i,vehicle_index,0,0])**2) + ((inp[i,j,0,1] - inp[i,vehicle_index,0,1])**2))
        
        included = []
        for j in range(6):
            min_distance = -1
            min_index = -1
            
            for k in range(len(distances)):
                if k == vehicle_index:
                    continue
                
                if min_index == -1 and k not in included:
                    min_distance = distances[k]
                    min_index = k
                elif k not in included and distances[k] < min_distance:
                    min_distance = distances[k]
                    min_index = k
            if min_index == -1:
                real_input[i][j] = np.zeros((19,4))
            else:
                included.append(min_index)
                real_input[i][j] = inp[i,min_index,:,:]
        real_input[i][6] = inp[i,vehicle_index,:,:]
    
    real_input = torch.LongTensor(real_input)
    offsets = torch.LongTensor(offsets)
    num_cars = torch.LongTensor(num_cars)
    return [real_input, scene_ids, offsets]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0, drop_last=True)

In [4]:
model = torch.nn.Sequential(
    torch.nn.Linear(76, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 60)
)
model.to(device)
if cuda_status:
    model = model.cuda()

### Visualize the batch of sequences

In [5]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm

agent_id = 0
epoch = 3

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out, scene_ids, track_ids, agent_ids = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])
        
# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epoch_count = 0

for i in trange(epoch):
    iterator = tqdm(val_loader)
    total = 0
    count = 0
    
    for i_batch, sample_batch in enumerate(iterator):
        inp, out, scene_ids, offsets = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        
        x = torch.flatten(inp, start_dim=2)

        x = x.float()
        y = out.float()

        if cuda_status:
            #model = model.cuda()
            #x = inp.cuda()
            #y = out.cuda()
            x.to(device)
            y.to(device)
            x = x.cuda()
            y = y.cuda()

        y_pred = None

        # Forward pass: predict y by passing x to the model.    
        y_pred = model(x)
        y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 7, 30, 2]))
        
        for i in range(y_pred.shape[0]):
            y_pred[i,:,:,0] += offsets[i][0]
            y_pred[i,:,:,1] += offsets[i][1]

        # Compute the loss.
        loss = loss_fn(y_pred[:,6,:,:], y[:,6,:,:])
        total += torch.sqrt(loss).item()

        # Before backward pass, zero outgradients to clear buffers  
        optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        loss.backward()

        # makes an gradient descent step to update its parameters
        optimizer.step()
        
        count += 1
        
        #print(torch.sqrt(loss).item(), end='\r')
        iterator.set_postfix(loss=total / count, curr=torch.sqrt(loss).item())
    
    epoch_count += 1
    torch.save(model, './models/new-linear' + str(epoch_count) + '.pt')


In [ ]:
torch.save(model, './models/3epochlinearreoriented.pt')

In [ ]:
model = torch.load('./models/6epochmodel.pt')
model.eval()
model.to(device)
if cuda_status:
    model = model.cuda()

In [18]:
import pandas as pd

# Submission output
writeCSV = True
val_path = "./new_val_in/new_val_in"

if writeCSV:
    
    dataset = ArgoverseDataset(data_path=val_path)
    test_loader = DataLoader(dataset,batch_size=64, shuffle = False, collate_fn=test_collate, num_workers=0)
    
    data = []
    
    with torch.no_grad():
        for i_batch, sample_batch in enumerate(tqdm(test_loader)):
            inp, scene_ids, offsets = sample_batch
            inp = torch.flatten(inp, start_dim=2)

            if cuda_status:
                model = model.cuda()
                x = inp.cuda()
            else:
                x = inp

            y_pred = None

            # Forward pass: predict y by passing x to the model.    
            y_pred = model(x.float())
            y_pred = torch.reshape(y_pred, torch.Size([64, 7, 30, 2]))
            
            for i in range(y_pred.shape[0]):
                y_pred[i,:,:,0] += offsets[i][0]
                y_pred[i,:,:,1] += offsets[i][1]
            
            for i in range(64):
                row = []
                row.append(scene_ids[i].item())
                
                curr = y_pred[i]
                
                for j in range(30):
                    row.append(str(curr[6][j][0].item()))
                    row.append(str(curr[6][j][1].item()))
                    
                data.append(row)

    df = pd.DataFrame(data, columns = ['ID','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','v20','v21','v22','v23','v24','v25','v26','v27','v28','v29','v30','v31','v32','v33','v34','v35','v36','v37','v38','v39','v40','v41','v42','v43','v44','v45','v46','v47','v48','v49','v50','v51','v52','v53','v54','v55','v56','v57','v58','v59','v60'])
    print(df)
    df.to_csv('submission.csv', index=False)
                
                
                

veh4
[46.10216554532715, 24.853427753405565, 32.28550953565024, 0]
1
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
veh6
[47.66296315422096, 34.423935338928786, 84.51925746723347, 37.2726834248287, 0, 1529.5755947320813]
1
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
5
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
veh6
[62.22617751371805, 73.14614411189082, 48.53822189136104, 0, 1412.9685506054266, 1415.9115438472843]
2
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
4
torch.Size([64, 7, 30, 2])
5
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
veh7
[18.436348778942868, 0, 1694.7229124550124, 1661.673328304935, 1690.0918614087223, 1697.523711174604, 1703.9189387996132]
0
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
4
torch.Si

0
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
veh5
[18.131595647989382, 0, 1482.7273012931273, 1465.1484566418517, 1432.2086963847134]
0
torch.Size([64, 7, 30, 2])
4
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
veh4
[62.45958458900643, 0, 1826.234240178406, 1847.9032036337835]
0
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
veh3
[24.347696045711903, 50.103959307436476, 0]
0
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
veh9
[64.2484154413068, 93.41731073688912, 22.116299572647982, 58.62305100704841, 19.36427374461149, 105.03604756969152, 45.75994331409704, 13.652885249454258, 0]
7
torch.Size([

8
torch.Size([64, 7, 30, 2])
9
torch.Size([64, 7, 30, 2])
7
torch.Size([64, 7, 30, 2])
6
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
5
torch.Size([64, 7, 30, 2])
veh8
[23.35236621442247, 0, 1771.2451834796898, 1702.6960239573004, 1696.8579050704275, 1718.6205951285467, 1727.5984776561943, 1711.5890277750673]
0
torch.Size([64, 7, 30, 2])
4
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
7
torch.Size([64, 7, 30, 2])
5
torch.Size([64, 7, 30, 2])
6
torch.Size([64, 7, 30, 2])
veh3
[15.787068659673157, 25.046615475495763, 0]
0
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
veh24
[49.31256931237208, 45.980435398690226, 84.38262986335666, 49.557593546549455, 51.70771219073611, 55.246800670943834, 53.01710550909489, 43.14936330597402, 51.370273790363186, 49.5013859102071, 59.38284076518864, 60.29071011824707, 51.416455104069065, 57.288054100118686,

veh15
[42.64581774904047, 11.803563527108098, 0, 1066.9895149437973, 1082.6778144951525, 1067.4147506944055, 1078.4174168660297, 1084.9867510711824, 1069.5920951465564, 1093.2575176965397, 1087.5647681862447, 1086.91144993509, 1096.5007979933257, 1097.6473135757224, 1094.7056111119555]
1
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
5
torch.Size([64, 7, 30, 2])
8
torch.Size([64, 7, 30, 2])
6
torch.Size([64, 7, 30, 2])
veh13
[21.332633165949503, 31.008920678106385, 41.26956064391996, 34.517352619167326, 12.146080961151368, 0, 1810.475282902255, 1742.5248204831976, 1755.8532114046436, 1811.4732954145363, 1772.5653020410841, 1739.1344111367587, 1745.050572333077]
4
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
11
torch.Size([64, 7, 30, 2])
veh4
[20.596819496919224, 15.295237499337263, 91.47349556360575, 0]
1
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7,

veh11
[8.898575966739838, 0, 1292.7799019941485, 1300.3400036144392, 1306.2062432862583, 1314.273040505663, 1308.8000229217603, 1296.7263589516488, 1278.428918634118, 1333.201691418069, 1262.0471465044402]
0
torch.Size([64, 7, 30, 2])
10
torch.Size([64, 7, 30, 2])
8
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
7
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
veh3
[8.117400855894296, 0, 2078.6310398913993]
0
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
veh12
[19.704313721823628, 0, 1701.5168233079567, 1720.9534421360736, 1738.0437997933193, 1711.8523592880315, 1685.2427124898063, 1700.183298941617, 1734.4264181567346, 1710.3986377450142, 1660.7512607250972, 1711.1789064852337]
0
torch.Size([64, 7, 30, 2])
10
torch.Size([64, 7, 30, 2])
6
torch.Size([64, 7, 30, 2])
7
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
9
torch.Size([64, 

...

veh17
[26.25523687531511, 33.212267123760164, 32.21008196020017, 26.94078748428556, 24.36026447622362, 31.0546825864776, 24.382703610378453, 37.46549519328585, 29.316459079316157, 42.89125397990539, 18.610581929795345, 0, 2342.3091170893736, 2310.8926846567324, 2319.1238647385785, 2286.8569041372048, 2286.7470345449233]
10
torch.Size([64, 7, 30, 2])
4
torch.Size([64, 7, 30, 2])
6
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
8
torch.Size([64, 7, 30, 2])
veh6
[9.077828328575395, 36.51588742150035, 5.952439957099197, 18.898321232589847, 0, 1364.1355779393778]
2
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
5
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
veh8
[14.73777341839775, 19.102271043862903, 49.45681759515694, 31.063990351885433, 40.251583091926335, 0, 1255.7029405874623, 1243.3888169032243]
0
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
3
torch.Size([64,

veh5
[36.26444856181033, 0, 2080.964079459326, 2072.902192579283, 2042.92039737235]
0
torch.Size([64, 7, 30, 2])
4
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
veh8
[30.990306692362996, 22.508091806666858, 33.0083089829525, 39.901265166834776, 24.351920914911286, 11.842149128079738, 0, 2337.3703172582645]
5
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
4
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
veh13
[62.17826760927507, 35.42795354770192, 43.62113135380532, 38.98315848552824, 48.46756123622789, 57.0641643034948, 53.527085907802324, 61.61712757144924, 30.943883495470473, 8.613983194895008, 40.297411178092446, 16.40141018642957, 0]
9
torch.Size([64, 7, 30, 2])
11
torch.Size([64, 7, 30, 2])
8
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
10
torch.Size([64, 7, 30, 2])
veh

0
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
5
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
4
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
veh11
[31.96481964322994, 39.62963609018666, 37.717947381684226, 9.290305911838795, 63.20366772016676, 22.923383396603047, 0, 1816.4094389756954, 1827.0811421499593, 1812.3803408777087, 1857.418638864163]
3
torch.Size([64, 7, 30, 2])
5
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
4
torch.Size([64, 7, 30, 2])
veh8
[142.49435981162551, 140.36449482062764, 132.56847434675032, 124.24662030111725, 193.58004642782788, 78.79634672884588, 0, 951.7526595707521]
5
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7, 30, 2])
4
torch.Size([64, 7, 30, 2])
veh5
[31.455595174363456, 0, 1360.2191918951887, 1370.8608153273622, 1374.8460368346705]
0
torch.Size([64, 7, 30, 2])
2
torch

torch.Size([64, 7, 30, 2])
5
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7, 30, 2])
6
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
veh4
[33.350093849427545, 12.27447928821728, 49.89454210663978, 0]
1
torch.Size([64, 7, 30, 2])
0
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
-1
torch.Size([64, 7, 30, 2])
veh12
[14.39188196447887, 20.11239475675541, 55.332841863326294, 42.59826931728712, 34.41521440325303, 39.713088453698106, 45.5455161675521, 26.156771528178773, 0, 1150.0826057288232, 1223.4325686362938, 1219.4908773746527]
0
torch.Size([64, 7, 30, 2])
1
torch.Size([64, 7, 30, 2])
7
torch.Size([64, 7, 30, 2])
4
torch.Size([64, 7, 30, 2])
5
torch.Size([64, 7, 30, 2])
3
torch.Size([64, 7, 30, 2])
veh9
[38.11840363016925, 61.33173755283638, 11.854970000835225, 27.909035196607366, 7.2230890617852985, 18.32723000398971, 0, 1335.6525558692276, 1395.9741222529879]
4
torch.Size([64, 7, 30, 2])
2
torch.Size([64, 7,

         ID                 v1                  v2                  v3  \
0     10002      1714.26171875   335.2650451660156  1711.9095458984375   
1     10015   726.574951171875  1229.9957275390625   726.0277709960938   
2     10019   574.285888671875   1243.830322265625   574.0863647460938   
3     10028   1689.58544921875   312.7535095214844  1687.9134521484375   
4      1003  2122.057861328125   677.7062377929688    2119.35888671875   
...     ...                ...                 ...                 ...   
3195   9897  256.1117858886719   805.9766235351562   256.0670471191406   
3196     99  587.6826782226562   1154.347412109375   587.4851684570312   
3197   9905  1756.657470703125   442.2682189941406  1754.7603759765625   
3198   9910  575.2328491210938  1288.1165771484375   574.8734130859375   
3199   9918  583.4385375976562   1164.526123046875   583.5875244140625   

                      v4                  v5                  v6  \
0      337.8272399902344  1718.048217773437